# Script Script.py

In [ ]:
#!/usr/bin/env python3

exec(open("Funções.py").read())
exec(open("Doc.py").read())
exec(open("Docx.py").read())
exec(open("Excel.py").read())

# Script Funções.py

In [1]:
#!/usr/bin/env python3

# Importação de bibliotecas
import pandas as pd
import io
import os
import csv
import copy
import textract
from docx import Document

In [2]:
# Cria a tabela padrão
def gera_tabela():
    tabela = [("Cafe da Manha","Cha"),
              ("Cafe da Manha","Pao"),
              ("Cafe da Manha","Fruta"),
              ("Almoco","Prato Principal"),
              ("Almoco","Guarnicao"),
              ("Almoco","Sobremesa"),
              ("Almoco","Saladas"),
              ("Almoco","Opcao Vegana"),
              ("Jantar","Prato Principal"),
              ("Jantar","Guarnicao"),
              ("Jantar","Saladas"),
              ("Jantar","Opcao Vegana"),
             ]
    tabela = pd.MultiIndex.from_tuples(tabela, names=["Refeicao","Itens"])
    tabela = pd.DataFrame(columns=tabela)
    return tabela;

In [3]:
# Encontra o ano e o mês do arquivo
def gera_data(file_name):
    file_name = file_name.split(" ")
    meses = ["erro",
             "janeiro",
             "fevereiro",
             "março",
             "abril",
             "maio",
             "junho",
             "julho",
             "agosto",
             "setembro",
             "outubro",
             "novembro",
             "dezembro",
            ]
    for i in range(0,len(file_name)):
        for j in range(1,len(meses)):
            if file_name[i].lower() == meses[j]:
                if j < 10:
                    return "0" + str(j)
                else:
                    return str(j)
    return meses[0]

In [4]:
# Função auxiliar da ordenação
def ordena(df,Refeição,lista):
    if lista:
        max = len(df)
    else:
        max = 1
    for i in range(0,max):
        if lista:
            string = df[i].split(" ")
        else:
            string = df.split(" ")
        for string_i in range(0,len(string)):
            if   string[string_i].lower() == "chá":
                return 0;
            elif string[string_i].lower() == "pão":
                return 1;
            elif string[string_i].lower() == "bife":
                if Refeição == "Almoco":
                    return 3;
                else:
                    return 8;
            elif string[string_i].lower() == "gelatina":
                    return 5;
            elif string[string_i].lower() == "alface":
                if Refeição == "Almoco":
                    return 6;
                else:
                    return 10;
            elif string[string_i].lower() == "pts":
                if Refeição == "Almoco":
                    return 7;
                else:
                    return 11;
    if   Refeição == "Cafe da Manha":
        return 2;
    elif Refeição == "Almoco":
        return 4;
    else:
        return 9;

In [5]:
# Verifica se o input pode ser representado como um inteiro
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [6]:
# Verifica se o input pode receber a função split()
def RepresentsStr(s):
    try: 
        s.split()
        return True
    except:
        return False

In [7]:
# Transforma docx em dataframe
def read_docx_tables(filename, tab_id=None, **kwargs):
    """
    parse table(s) from a Word Document (.docx) into Pandas DataFrame(s)

    Parameters:
        filename:   file name of a Word Document

        tab_id:     parse a single table with the index: [tab_id] (counting from 0).
                    When [None] - return a list of DataFrames (parse all tables)

        kwargs:     arguments to pass to `pd.read_csv()` function

    Return: a single DataFrame if tab_id != None or a list of DataFrames otherwise
    """
    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        return pd.read_csv(vf, **kwargs)

    doc = Document(filename)
    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        try:
            return read_docx_tab(doc.tables[tab_id], **kwargs)
        except IndexError:
            print('Error: specified [tab_id]: {}  does not exist.'.format(tab_id))
            raise

In [8]:
# Function to insert row in the dataframe 
def Insert_row(row_number, df, row_value): 
    # Starting value of upper half 
    start_upper = 0
   
    # End value of upper half 
    end_upper = row_number 
   
    # Start value of lower half 
    start_lower = row_number 
   
    # End value of lower half 
    end_lower = df.shape[0] 
   
    # Create a list of upper_half index 
    upper_half = [*range(start_upper, end_upper, 1)] 
   
    # Create a list of lower_half index 
    lower_half = [*range(start_lower, end_lower, 1)] 
   
    # Increment the value of lower half by 1 
    lower_half = [x.__add__(1) for x in lower_half] 
   
    # Combine the two lists 
    index_ = upper_half + lower_half 
   
    # Update the index of the dataframe 
    df.index = index_ 
   
    # Insert a row at the end 
    df.loc[row_number] = row_value 
    
    # Sort the index labels 
    df = df.sort_index() 
   
    # return the dataframe 
    return df 

In [9]:
# Corrige os casos particulares
def casos_particulares(df,caso):
    if caso == "05 Cardapio maio 2019 RU - FINAL com símbolos e descrições preparações.docx":
        df.iloc[1,2] = "01\nChá Maçã\nPão hot dogc/ doce de banana\n Laranja"
        df.iloc[1,3] = "02\nChá de morango\nPão francês presunto\nBanana"
        df.iloc[1,4] = "03\nChá de erva doce\nPão hot dog com pate de azeitona\nSalada de frutas"
        df.iloc[1,5] = "04\nChá de hortelã\nPão francês com queijo\nMamão"
        df.iloc[1,6] = "05\nChá de erva cidreira\nPão francês com presunto\nBanana"
    elif caso == "CARDÁPIO MARÇO 2019 - simbolos.docx":
        for i in range(0,4):
            df.iloc[0,i] = "n\no\np\ne"
        df.iloc[6,1] = "Bife ao molho de bacon e cenoura \nSopa Califórnia  (frango, batata, cenoura e abobrinha)\nAlmeirão e tomate\nHambúrguer de feijão branco ao sugo"
    elif caso == "04 Cardápio abril 2019 RU Versão Definitiva com resposta Blu símbolos.docx":
        df.iloc[1,6] = "07\nChá de erva-doce\nPão hot dog com \nDoce de abóbora/Salada de frutas\n"
    elif caso == "Cardápio abril 2018 FINAL USAR ESTE.docx":
        df.drop("DOMINGO.1",axis=1,inplace=True)
        df.iloc[3,1] = "Frango ao m. sugo– assado com alecrim (RUC)\nSopa de feijão\nEscarolae cenoura ralada\nPanqueca de grão-de-bico"
        df.iloc[11,0] = "23\nChá de hortelã\nPão hotdog com doce\nBanana/Melão\n"
        df.iloc[12,0] = "Guisadinho (moída c/ milho e ervilha)\nEspaguete ao alho e óleo Maçã\nNaN\nRepolho bicolor e cenoura ral.\nGuisadinho de PTS (moída c/ milho)"
        df.iloc[15,6] = "Barreado\nSopa abóbora\nNaN\nAcelga e tomate\nLentilha c/ legumes"
    elif caso == "01 Cardápio Março 2018 blum. revisado final.docx":
        df.iloc[3,4] = "Quibe \nSopa juliana\nAcelga e beterraba ralada\nQuibe de PTS"
        df.iloc[5,5] = "Filé de frango ao molho de mostarda\nLegumes refogados\nPudim de chocolate\nAlmeirão e tomate\nBOLINHO DE LENTILHA"
        df.iloc[9,4] = "Peixe à milanesa c/ limão\nPirão de peixe\nBanana\nAlface americana e cenoura ral.\nHambúrguer de soja"
        df.iloc[10,2] = "Bife à rolê\nSopa de mandioca\nAcelga e beterraba ralada\nBerinjela rolê"
        df.iloc[11,4] = "30\nChá de hortelã \nPão francês com doce de abóbora \nMelão"
        df.iloc[12,4] = "PEIXE NO FUBÁ AO MOLHO TÁRTARO\nBATATA AMASSADA\nBanana \nAcelga e beterraba ralada\nCreme de ervilha"
        df.iloc[13,5] = "Empadão de frango\nCaldo italiano\nAlface e rabanete\nEmpadão de legumes c/ feijão branco"
    elif caso == "Cardápio de setembro 2018 revisão almoço e jantar2.docx":
        df.iloc[8,3] = "20\nChá de erva-cidreira\nPão hot dog com doce de banana\nMaca"
    elif caso == "cardápio junho 2018 RU Revisado almoço Lu e vaneise.docx":
        df.iloc[2,4] = "Frango ao molho sugo (coxa/sob)\nPolenta cremosa\nMamão picado\nAlface crespa e pepino\nFeijão branco ao sugo"
    elif caso == "Cardápio de julho 2018 almoço e jantar revisado.docx":
        df.drop("DOMINGO.1",axis=1,inplace=True)
        df.iloc[1,0] = "02\nChá de hortelã\nPão hotdog com queijo\nMaçã/Ponkan"
        df.iloc[6,3] = "Panqueca de Carne moída\nCaldo Italiano\nRúcula e mista (cenoura, vagem, ab ) cozida \nPanqueca de P.T.S"
        df.iloc[12,4] = "Coxa/sobrecoxa ao molho curry\nSeleta de legumes\nNaN\nAlface crespa e pepino\nHambúrguer de soja"
        df.iloc[15,0] = "Picadinho de carne\nAbobrinha colorida \nSagú de vinho\nAcelga e cenoura ralada\nFeijão cavalo refogado"
        df.iloc[15,6] = "Strogonoff de carne\nPurê de batata\nNaN\nAcelga e tomate\nGrão de bico com champignon"
    elif caso == "Cardápio novembro 18 - revisado almoço e jantar FINAL 3.docx":
        df.drop("DOMINGO.1",axis=1,inplace=True)
        df.iloc[12,0] = "Guisadinho (moída c/ milho e ervilha)\nEspaguete ao alho e óleo\nMaçã\nRepolho bicolor e cenoura ralada\nGuisadinho de PTS c/ milho e ervilha"
        df.iloc[14,1] = "27\nChá de camomila\nPão francês com presunto\nMelancia"
        df.iloc[15,6] = "Strogonoff\nBatata chips\nNaN\nAcelga e tomate\nCharutinho de repolho com lentilha"
    elif caso == "Cardápio outubro 2018 FINAL definitivo.docx":
        df.iloc[1,6] = "07\nChá mate\nPão hot dog com doce de abóbora\nMelão"
        df.iloc[5,4] = "12\nChá de hortelã\nPãofrancês com presunto\nAbacaxi"
        df.iloc[7,3] = "Panqueca de carne\nCaldo italiano\nRadite e Chuchu c/ milho\nPanqueca(cenoura e ervilha seca) (era de brócolis, mas já tem no dia anterior)\n"
    elif caso == "cardápio maio RU revisado Lucyanne vaneise.docx":
        df.iloc[5,2] = "Carne moída refogada\nBolinho de cenoura\nPudim Prestígio\nEscarola e abobrinha\nTomate recheado c/ feijão branco temperado com cebola e tempero verde\n"
        df.iloc[5,3] = "Peixe frito no fubá c/ limão\nJardineira de legumes\nGelatina c/ creme\nAcelga e beterraba ralada\nBolinho de lentilha\n"
        df.iloc[5,4] = "Strogonoff de carne\nBatata chips\nLaranja\nAlface americana e tomate\nPimentão recheado com PTS, arroz e tomate\n"
        df.iloc[6,3] = "Bife à role\nSopa de batata-salsa\nAlface americana e vagem\nBerinjela à rolêcom pasta de feijão branco\n"
        df.iloc[9,1] = "Almôndega à chinesa\nCreme de espinafre/milho\nGelatina de cereja\nAgrião e rabanete\nQuibe de PTS\n"
        df.iloc[9,4] = "Picadinho de carne\nBolinho de arroz\nSagu de vinho\nAcelga e abobrinha ralada\nPicadinho vegano (pts ao sugo temperada)\n"
    return df

# Script Doc.py

In [10]:
#!/usr/bin/env python3

# Coletando o texto
text = textract.process("../2019/Cardápio fevereiro 2019 versão final REVISADA BLUME com breves descrições.doc")
text = text.decode("utf-8") 
text = "SEGUNDA\nTERÇA\nQUARTA\nQUINTA\nSEXTA\nSÁBADO\nDOMINGO\n25\nChá de erva cidreira\nPão hot dog com margarina\nMaçã\n26\nChá de camomila\nPão francês com presunto\nMelancia\n27\nChá Maçã\nPão hot dog c/ doce de abóbora\nLaranja\n28\nChá de morango\nPão francês com patê de frango\nBanana\n01\nChá de erva doce\nPão hot dog com margarina\nAbacaxi\n02\nChá de hortelã\nPão francês com queijo\nMamão\n03\nChá de erva cidreira\nPão francês com presunto\nBanana\nBisteca ao m. do cheff (molho escuro com batata salsa)\nQuirera com costelinha\nLaranja\nAlface americana e beterraba ralada\nSoja refogada\nBife de panela\nMacarrão com brócolis\nMelão\nAcelga e cenoura ralada\nFeijão cavalo ao alho e óleo\nFilé de frango à milanesa\nAbobrinha refogada\nCreme de abacaxi\nAlmeirão e tomate\nTorre de berinjela (com ervilha seca)\nCarne moída ao sugo\nPurê prático\nSagu de vinho\nRepolho e beterraba ralada\nTrigo em grão c/ PTS ao sugo\nFrango assado/frito\nQuibebe\nGelatina miscelânea\nRadite e Berinjela ao vinagrete\nHambúrguer de soja\nTirinhas de carne ao molho oriental\nLegumes refogado\nBanana\nAlface e brócolis\nBolinho de lentilha\nChurrasco\nFarofa úmida\nGelflan\nAgrião e maionese de legumes\nTorre de berinjela\nPicadinho especial (molho com cenoura, tomate e milho)\nSopa Califórnia (frango, batata, cenoura e abobrinha)\nAcelga e tomate\nCharutinho de repolho com lentilha\nFilé de frango ao molho de limão\nSopa leve (paleta bovina, batata cenoura e arroz)\nAlmeirão e Natural\nHamb g-bico c/ abóbora\nPanqueca de carne moída\nSopa de legumes com arroz\nRepolho bicolor e vagem\nPanqueca de triguilho\nFrango à paulista (molho escuro com bacon, tomate e pimentão verde)\nSopa de mandioca\nRadite e cenoura ralada\nPimentão recheado com grão-de-bico\nStrogonoff de carne\nBatata chips\nAlface e chuchu com milho verde\nStrogonoff de feijão branco\nRocambole de carne\nCanja (frango, batata, cenoura, arroz)\nAgrião e abobrinha cozida\nPolpeta de PTS recheada c/ cenoura e vagem\nFrango americano\nCreme rio branco (creme de batata salsa com couve)\nEscarola e brócolis c/ c-flor\nCroquete de trigo em grão"
text = text.split("\n")

In [11]:
# Gerando dataframe
dias = []
for i in range(0,7):
    dias += [text[i]]
dias
df = pd.DataFrame([[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0],[0,0,0,0,0,0,0]],columns=dias)
for i in [0,1,2,3,4,5,6]:
    for j in [1,2,3,4]:
        df.iloc[j-1,i] = text[6+(i*4)+j]
        df.iloc[j-1+4,i] = text[34+(i*5)+j]
        df.iloc[j-1+9,i] = text[69+(i*4)+j]
    df.iloc[5-1+4,i] = text[34+(i*5)+5]

In [12]:
data = "2019-02"
tabela = gera_tabela()
r,c = df.shape
maior = 0
# Criando tabela csv
for i in range(0,r):
    for j in range(0,c):
        if RepresentsInt(df.iloc[i,j]):
            if int(df.iloc[i,j]) > maior:
                Itens = []
                for k in range(1,13):
                    Itens += [df.iloc[i+k,j]]
                for u,k in [("Cafe da Manha",[0,1])]:
                    organizado = False
                    for v in k:
                        while not organizado:
                            w = copy.copy(ordena(Itens[v],u,False))
                            if Itens[v] == Itens[w]:
                                organizado = True
                            else:
                                Itens[v],Itens[w] = Itens[w],Itens[v]
                tabela.loc[data + "-" + str(df.iloc[i,j])] = Itens
                maior = int(df.iloc[i,j])

# Reordenando a tabela
for i,k in [("Cafe da Manha",[0,1]),("Almoco",[3,4,5,6,7]),("Jantar",[8,9,10,11])]:
    for j in k:
        a = copy.copy(tabela.iloc[:,j])
        b = copy.copy(tabela.iloc[:,ordena(a,i,True)])
        tabela.iloc[:,j],tabela.iloc[:,ordena(a,i,True)] = copy.copy(b),copy.copy(a)

# Criando pasta e arquivo csv
newpath = "../2019_csv"
if not os.path.exists(newpath):
    os.makedirs(newpath)
tabela.to_csv(newpath + "/" + data + ".csv")

# Script Docx.py

In [13]:
#!/usr/bin/env python3

# Criando DataFrame com os arquivos
files = os.listdir("..")
arquivos = pd.DataFrame()
for i in range(0,len(files)):
    if RepresentsInt(files[i]):
        arquivos[files[i]] = []
for l in arquivos.columns:
    files = os.listdir("../" + l)
    for i in range(0,len(files)):
        if files[i].split(".")[-1] == "docx":
            arquivos = arquivos.append({l: files[i]}, ignore_index=True)

In [14]:
ano = arquivos.columns.tolist()
for ano_i in range(0,len(ano)):
    l = arquivos[str(ano[ano_i])].dropna().tolist()
    bb = []
    for arquivos_i in range(0,len(l)):
        df = read_docx_tables("../" + ano[ano_i] + "/" + l[arquivos_i])
        if len(df) == 1:
            df = df[0]
        elif len(df) == 2:
            df = df[0].append(df[1],ignore_index=True)
            df = df[["SEGUNDA","TERÇA","QUARTA","QUINTA","SEXTA","SÁBADO","DOMINGO"]]
        df = Insert_row(0,df,df.columns)
        aa = {}
        for k in range(0,len(df.index)):
            c = []
            for i in range(0,len(df.columns)):
                a = df.iloc[k,i]
                if type(a) == str:
                    a = a.split("\n")
                    b = []
                    for j in range(0,len(a)):
                        if a[j] != "" and a[j] != " ":
                            b += [a[j]]
                    c += [len(b)]
                    c1 = pd.DataFrame(pd.DataFrame(c)[0].value_counts())
                    c1 = c1[c1[0] == max(c1[0])].index[0]
            if len(list(set(c))) > 1:
                c = pd.DataFrame(c)[pd.DataFrame(c)[0] != c1]
                for j in range(0,len(c.index)):
                    aa[k,c.index[j-1]] = c.iloc[j,0],c1
        df = casos_particulares(df,l[arquivos_i])
        k = 0
        while k <= df.index[-1]:
            ii = df.index[-1]
            for i in range(0,len(df.columns)):
                a = df.iloc[k,i]
                if RepresentsStr(a):
                    a = a.split("\n")
                    b = []
                    for j in range(0,len(a)):
                        if a[j] != "":
                            b += [a[j]]
            for i in range(ii + 1,ii + len(b)):
                df.loc[i,:] = 0
            for i in range(ii,k,-1):
                df.loc[i+len(b)-1,:] = df.loc[i,:]
            for i in range(1,len(b)):
                df.loc[k+i,:] = 0

            for i in range(0,len(df.columns)):
                a = df.iloc[k,i]
                if RepresentsStr(a):
                    a = a.split("\n")
                    b = []
                    for j in range(0,len(a)):
                        if a[j] != "":
                            b += [a[j]]
                    df.iloc[k:k+len(b),i] = b
            k += len(b)
        data = ano[ano_i] + "-" + gera_data(l[arquivos_i])
        tabela = gera_tabela()
        r,c = df.shape
        maior = 0
        # Criando tabela csv
        for i in range(0,r):
            for j in range(0,c):
                if RepresentsInt(df.iloc[i,j]):
                    if int(df.iloc[i,j]) > maior:
                        Itens = []
                        for k in range(1,13):
                            Itens += [df.iloc[i+k,j]]
                        for u,k in [("Cafe da Manha",[0,1])]:
                            organizado = False
                            for v in k:
                                while not organizado:
                                    w = copy.copy(ordena(Itens[v],u,False))
                                    if Itens[v] == Itens[w]:
                                        organizado = True
                                    else:
                                        Itens[v],Itens[w] = Itens[w],Itens[v]
                        tabela.loc[data + "-" + str(df.iloc[i,j])] = Itens
                        maior = int(df.iloc[i,j])

        # Reordenando a tabela
        for i,k in [("Cafe da Manha",[0,1]),("Almoco",[3,4,5,6,7]),("Jantar",[8,9,10,11])]:
            for j in k:
                a = copy.copy(tabela.iloc[:,j])
                b = copy.copy(tabela.iloc[:,ordena(a,i,True)])
                tabela.iloc[:,j],tabela.iloc[:,ordena(a,i,True)] = copy.copy(b),copy.copy(a)

        # Criando pasta e arquivo csv
        newpath = "../" + str(ano[ano_i]) + "_csv"
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        tabela.to_csv(newpath + "/" + data + ".csv")

# Script Excel.py

In [15]:
#!/usr/bin/env python3

# Criando DataFrame com os arquivos
files = os.listdir("..")
arquivos = pd.DataFrame()
for i in range(0,len(files)):
    if RepresentsInt(files[i]):
        arquivos[files[i]] = []
for l in arquivos.columns:
    files = os.listdir("../" + l)
    for i in range(0,len(files)):
        if files[i].split(".")[-1] == "xlsx":
            arquivos = arquivos.append({l: files[i]}, ignore_index=True)

In [16]:
ano = arquivos.columns.tolist()
for ano_i in range(0,len(ano)):
    l = arquivos[str(ano[ano_i])].dropna().tolist()
    for arquivos_i in range(0,len(l)):
        xls_file = pd.ExcelFile("../" + str(ano[ano_i]) + "/" + l[arquivos_i])
        df = xls_file.parse(xls_file.sheet_names[0])
        data = ano[ano_i] + "-" + gera_data(l[arquivos_i])
        tabela = gera_tabela()
        r,c = df.shape
        maior = 0
        
        # Criando tabela csv
        for i in range(0,r):
            for j in range(0,c):
                if type(df.iloc[i,j]) == int:
                    if df.iloc[i,j] > maior:
                        Itens = []
                        for k in range(1,13):
                            Itens += [df.iloc[i+k,j]]
                        for u,k in [("Cafe da Manha",[0,1])]:
                            organizado = False
                            for v in k:
                                while not organizado:
                                    w = copy.copy(ordena(Itens[v],u,False))
                                    if Itens[v] == Itens[w]:
                                        organizado = True
                                    else:
                                        Itens[v],Itens[w] = Itens[w],Itens[v]
                        tabela.loc[data + "-" + str(df.iloc[i,j])] = Itens
                        maior = df.iloc[i,j]
        
        # Reordenando a tabela
        for i,k in [("Cafe da Manha",[0,1]),("Almoco",[3,4,5,6,7]),("Jantar",[8,9,10,11])]:
            for j in k:
                a = copy.copy(tabela.iloc[:,j])
                b = copy.copy(tabela.iloc[:,ordena(a,i,True)])
                tabela.iloc[:,j],tabela.iloc[:,ordena(a,i,True)] = copy.copy(b),copy.copy(a)
        
        # Criando pasta e arquivo csv
        newpath = "../" + str(ano[ano_i]) + "_csv"
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        tabela.to_csv(newpath + "/" + data + ".csv")

# Leitura das Tabelas

Rodar ```pd.read_csv("../2018_csv/2018-02.csv", header=[0,1], index_col=[0])``` para obter a mesma tabela

In [ ]:
pd.read_csv("../2018_csv/2018-02.csv", header=[0,1], index_col=[0])